In [1]:
from pycbc.waveform import td_approximants, get_td_waveform
from pycbc.filter import match
from pycbc.psd import aLIGOZeroDetHighPower

import numpy
import pylab

In [ ]:
f_low = 30
sample_rate = 2 * 4096

matchVals = []

for n in xrange(10, 101):
#     print (n)
    m1 = 100.0 / (n+1)
    m2 = n * 100.0 / (n+1)

    hp, hc = get_td_waveform(approximant='HGimri',
                             mass1=m1,
                             mass2=m2,
                             f_lower=f_low,
                             delta_t=1.0/sample_rate)

    sp, sc = get_td_waveform(approximant='SEOBNRv4',
                             mass1=m1,
                             mass2=m2,
                             f_lower=f_low,
                             delta_t=1.0/sample_rate)



    # Resize the waveforms to the same length
    tlen = max(len(sp), len(hp))
    sp.resize(tlen)
    hp.resize(tlen)

    # Generate the aLIGO ZDHP PSD
    delta_f = 1.0 / sp.duration
    flen = tlen/2 + 1
    psd = aLIGOZeroDetHighPower(flen, delta_f, f_low)

    # Note: This takes a while the first time as an FFT plan is generated
    # subsequent calls are much faster.
    m, i = match(hp, sp, psd=psd, low_frequency_cutoff=f_low)
    matchVals.append(m)

pylab.plot(xrange(10, 101), numpy.array(matchVals), label="Match value")

pylab.ylabel('Match')
pylab.xlabel('Eccentricity 2')
pylab.legend(loc='upper right')
pylab.show()